# Preprocesamiento del texto



In [1]:
import pandas as pd #armar y guardar el CSV
import re #Limpieza
import unicodedata #Normaliza caracteres Unicode.
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords # Recursos de NLTK (stopwords)
stop_en = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



**Limpieza clásica para texto (GitHub)**
    - lower
    - quitar URLs
    - quitar acentos (por seguridad)
    - dejar letras/espacios
    - quitar stopwords

In [2]:
def limpiar_texto(texto):
    #Pasa a minúsculas
    t = str(texto).lower() 

    # URLs
    t = re.sub(r"http\S+|www\S+", " ", t)

    # Acentos (por si hay)
    t = unicodedata.normalize("NFD", t)
    t = t.encode("ascii", "ignore").decode("utf-8")

    # Solo letras
    t = re.sub(r"[^a-z\s]", " ", t)

    # Espacios extra
    t = re.sub(r"\s+", " ", t).strip()

    # Stopwords
    palabras = [w for w in t.split() if w not in stop_en]
    return " ".join(palabras)

Definir repos a procesar

In [3]:
repos_slugs = ["auto1111_webui", "ytdlp"]

**Iteración por repositorio**

* Recorre cada slug en repos_slugs para procesar los repos de forma independiente.

**Carga de datos**

* Lee el archivo messages_raw.csv de cada repositorio.

* Contiene issues y comentarios sin limpiar.

**Construcción del texto base**

* Rellena valores nulos en title y body.

* Une title y body en una sola columna text.

**Limpieza del texto**

* Aplica la función limpiar_texto:

1. pasa a minúsculas

2. elimina URLs y acentos

3. conserva solo letras y espacios

4. elimina stopwords en inglés

* El resultado se guarda en text_clean.

**Eliminación de textos vacíos**

* Se eliminan filas donde text_clean queda vacío. (Paso clave para evitar errores en TF-IDE)

**Control de cambios**

* Muestra cuántos registros había antes y después de la limpieza.

**Guardado del resultado**

* Guarda el archivo final como messages_clean.csv.

* Dataset listo para análisis NLP posteriores.

In [4]:
for slug in repos_slugs:
    ruta_in = f"../data/{slug}/messages_raw.csv" #ruta donde está el archivo generado en el file 1
    df = pd.read_csv(ruta_in) #Carga el archivo en un DataFrame
 
    # texto base = title + body - evitar valores nulos
    df["title"] = df["title"].fillna("")
    df["body"]  = df["body"].fillna("") #Esto reemplaza NaN por cadena vacía
    df["text"] = (df["title"].astype(str) + " " + df["body"].astype(str)).str.strip()
    #Cada fila ahora tiene una columna text que contiene todo el contenido textual del mensaje
    
    # limpiar - llama a la función
    df["text_clean"] = df["text"].apply(limpiar_texto)

    # eliminar vacíos - importante para TF-IDF)
    antes = df.shape[0] #Guarda cuántas filas había antes
    df = df[df["text_clean"].str.strip() != ""].copy() #quita filas que esten vacías
    despues = df.shape[0] #Guarda cuántas filas quedaron

    print(f"\nRepo {slug}: antes={antes}, después={despues}, eliminados={antes-despues}")

    ruta_out = f"../data/{slug}/messages_clean.csv" #Guarda el nuevo archivo limpio
    df.to_csv(ruta_out, index=False)
    print("Guardado", ruta_out)


Repo auto1111_webui: antes=475, después=459, eliminados=16
Guardado ../data/auto1111_webui/messages_clean.csv

Repo ytdlp: antes=697, después=697, eliminados=0
Guardado ../data/ytdlp/messages_clean.csv
